In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import tensorflow as tf
import numpy as np
np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv", sep = ",")
df = df.dropna()
df.head()

In [ ]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"], infer_datetime_format=True, unit="s")
df = df.set_index("Timestamp")
df = df.drop(["Low", "High", "Volume_(BTC)", "Weighted_Price"], axis=1)
df.head()

In [ ]:
df = df.reindex(columns=["Open", "Close", "Volume_(Currency)"])
df.head()

In [ ]:
df["Open"] = df["Open"].resample("1H").first()
df["Close"] = df["Close"].resample("1H").last()
df["Volume_(Currency)"] = df["Volume_(Currency)"].resample("1H").sum()
df = df.dropna()
df.head()

In [ ]:
df = df.iloc[-int((df.shape[0]/2)):]
print(df.head(10))

In [ ]:
dataset = df.values
print(dataset[:10])
print(dataset.shape)

In [ ]:
mean = np.mean(dataset, axis=0)
stddev = np.std(dataset, axis=0)
dataset = (dataset - mean) / stddev
print(dataset[:10])
print(dataset.shape)

In [ ]:
def split_sequence(sequence, n_steps):
#function that splits a dataset sequence into input data and labels
    X, Y = [], []
    for i in range(sequence.shape[0]):
        if (i + n_steps) >= sequence.shape[0]:
            break
      # Divide sequence between data (input) and labels (output)
        seq_X, seq_Y = sequence[i: i + n_steps], sequence[i + n_steps, -2]
        X.append(seq_X)
        Y.append(seq_Y)
    return np.array(X), np.array(Y)
# Create training and validation datasets
dataset_size = dataset.shape[0]
x_train, y_train = split_sequence(dataset[0: math.ceil(0.7 * dataset_size)], 24)
x_val, y_val = split_sequence(dataset[math.floor(0.7 * dataset_size):], 24)
x_train.shape[-1]

In [ ]:
batch_size = 256
buffer_size = x_train.shape[0]
# Provide an infinite dataset
train_iterator = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size).batch(batch_size).repeat()
# Provide an infinite dataset
val_iterator = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size).repeat()
n_steps = x_train.shape[-2]
n_features = x_train.shape[-1]
# Define the model
model = Sequential()
model.add(Bidirectional(LSTM(64, activation="relu", input_shape=(n_steps, n_features))))
model.add(Dense(1))
# Compile the model
model.compile(optimizer="adam", loss="mse")
epochs = 1
steps_per_epoch = 800
validation_steps = 80
# Train with an infinite dataset
history = model.fit(train_iterator, epochs = epochs, steps_per_epoch = steps_per_epoch, validation_data = val_iterator, validation_steps = validation_steps)
print('=======================')
model.summary()